In [2]:
def lbp(img):
    
    qtdeLinhas, qtdeColunas = img.shape
    out = np.zeros((qtdeLinhas,qtdeColunas), dtype=int)
    
    for i in range(1,qtdeLinhas-1):
        for j in range(1,qtdeColunas-1):
            a=img[i-1,j]
            b=img[i-1,j+1]
            c=img[i,j+1]
            d=img[i+1,j+1]
            e=img[i+1,j]
            f=img[i+1,j-1]
            g=img[i,j-1]
            h=img[i-1,j-1]
            centro=img[i,j]
            
            soma=0
            
            if(a<centro):
                soma = soma + 2**7
            if(b<centro):
                soma = soma + 2**6
            if(c<centro):
                soma = soma + 2**5
            if(d<centro):
                soma = soma + 2**4
            if(e<centro):
                soma = soma + 2**3
            if(f<centro):
                soma = soma + 2**2
            if(g<centro):
                soma = soma + 2**1
            if(h<centro):
                soma = soma + 2**0

            out[i,j]=soma  
            
    return out

def histograma(img):
    qtdeLinhas, qtdeColunas= img.shape
    w = np.zeros(256)
    
    for i in range(qtdeLinhas):
        for j in range(qtdeColunas):
            w[img[i,j]] = w[img[i,j]]+1
    for i in range(256):
        w[i] = w[i]/(qtdeLinhas*qtdeColunas)
        
    return w

In [3]:
import cv2
cv = cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import time
from sklearn.ensemble import RandomForestClassifier
start_time = time.time()

# Função para facilitar a visualização das imagens
def showit(img, openCV=0):
    
    if(openCV):
        cv.imshow("t", img[::4,::4])
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    else:  
        # Para imagens preto e branco
        if(len(img.shape) < 3):
            plt.gray()
            plt.imshow(img)
            plt.show()
        else:
            plt.imshow(img[:,:,::-1])
            plt.show()

# Essa função realça as bordas da imagem
def improveEdges(imgCol):
    
    # Detecção de bordas
    sobelVert = cv.Sobel(imgCol, cv.CV_8U, 1, 0, ksize=3)
    sobelHori = cv.Sobel(imgCol, cv.CV_8U, 0, 1, ksize=3)
    imgColSobel = cv.add(sobelHori, sobelVert)
    
    # Realçando as bordas encontradas na imagem colorida
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (2,2))
    imgColSobel = cv.morphologyEx(imgColSobel, cv.MORPH_CLOSE, stt)
    
    # Adicionar as bordas encontradas na imagem colorida
    imgCol = cv.add(imgCol, imgColSobel)
    # showit(imgCol, 1)
    
    return imgCol


# Essa função é usada no treinamento do knn
def findSingleCoin(imgCol):
    
# ETAPA 1: ENCONTRAR A REGIÃO DE INTERESSE
    
    # Reduz tamanho da imagem
    # DOTO: Auto resize
    d = 850 / imgCol.shape[1]
    dim = (850, int(imgCol.shape[0] * d))
    imgCol = cv2.resize(imgCol, dim, interpolation=cv2.INTER_AREA)
        
    # Converter para tons de cinza
    imgGray = cv.cvtColor(imgCol, cv.COLOR_BGR2GRAY)

    # Encontrar a região de interesse (mascara da região da moeda)
    limiar, imgGrayOTSU = cv.threshold(imgGray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU) # limiar de otsu inclui as sombras
    ret,imgGray = cv.threshold(imgGray,(limiar-limiar//7) ,255,cv.THRESH_BINARY_INV) # 85% do limiar de otsu

    #showit(imgGrayOTSU,1)

    # Parâmetros para selecionar só a região de interesse
    val = 3
    maskSize=3
    
    # Remover pequenas falhas antes de dilatar
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (maskSize,maskSize))
    imgBin = cv.erode(imgGray, stt, iterations=1)
    # showit(imgBin, 1)

    # Fechar buracos na estrutura encontrada
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (maskSize,maskSize))
    imgBin = cv.morphologyEx(imgBin, cv.MORPH_CLOSE, stt)
    imgBin = cv.dilate(imgBin, stt, iterations=val+10)

    # showit(imgBin, 1)

    # Erosão para remover pequenas falhas e separar moedas coladas
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (maskSize,maskSize))
    imgBin = cv.erode(imgBin, stt, iterations=val)

    #showit(imgBin, 1)

    # Convertendo imgGray para BGR para poder realizar o bitwise
    imgGray = cv.cvtColor(imgBin, cv.COLOR_GRAY2BGR)

    # Selecionando região de interesse na imagem colorida com a máscara encontrada
    imgCol = cv.bitwise_and(imgCol, imgGray)

    #showit(imgCol, 1)
    
# ETAPA 2: REALÇAR AS CARACTERÍSTICAS DA ROI (Bordas e cores)
    imgCol = improveEdges(imgCol)
    
# ETAPA 3: DETECTAR AS MOEDAS

    # Achar Contornos das moedas
    moedas, hierarquia = cv.findContours(imgBin, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    # Quantidade de moedas encontradas
#     print(len(moedas), "moedas")
    
    # Para cada moeda encontrada
    for moeda in moedas:
        
        area = cv.contourArea(moeda)
        
        if(area < 10000.0):
            continue

        # Encontre o menor retângulo envolvente
        x, y, w, h = cv.boundingRect(moeda)

        # Desenhar o retângulo
        # cv.rectangle(imgCol, (x,y), (x+w, y+h), (0,255,0), 5)
        
        # Nova imagem com o menor retângulo envolvente da moeda
        imgCoinRoi = imgCol[y:y+h, x:x+w, ::-1]
                
    return imgCoinRoi

def calcHist(imag, roiDefined=False):   
    
    # Find the coin # PROBLEMA
    if(not roiDefined):
        imag = findSingleCoin(imag)
  
#     b = cv.calcHist([imag[:,:,0]],[0],None,[256],[0,256])
#     b = cv.normalize(b,b).flatten()
    
#     g = cv.calcHist([imag[:,:,1]],[0],None,[256],[0,256])
#     g = cv.normalize(g,g).flatten()
    
#     r = cv.calcHist([imag[:,:,2]],[0],None,[256],[0,256])
#     r = cv.normalize(r,r).flatten()
    
#     result = []
#     result.append(b.ravel().tolist())
#     result.append(g.ravel().tolist())
#     result.append(r.ravel().tolist())

    imag = cv.cvtColor(imag, cv.COLOR_BGR2GRAY)
    imag = lbp(imag)
#         r = cv.calcHist([imag],[0],None,[256],[0,256])
#         r = cv.normalize(r,r).flatten()
    r = histograma(imag)
    
#     # Flat the list
#     result = [y for x in result for y in x]

    return r

def calcHistFromFile(file):
    img = cv2.imread(file)
    return calcHist(img)

class Enum(tuple):__getattribute__ = tuple.index

# ETAPA 1: EXTRAIR CARACTERÍSTICAS DAS IMAGENS
Coins = Enum(("CINCO", "DEZ", "VINTECINCO", "CINQUENTA", "REAL"))

# Lista com o nome das imagens nesse diretório
sample_images_5_back = glob.glob("moedas/5/back/*")
sample_images_5_front = glob.glob("moedas/5/front/*")

sample_images_10_back = glob.glob("moedas/10/back/*")
sample_images_10_front = glob.glob("moedas/10/front/*")

sample_images_25_back = glob.glob("moedas/25/back/*")
sample_images_25_front = glob.glob("moedas/25/front/*")

sample_images_50_back = glob.glob("moedas/50/back/*")
sample_images_50_front = glob.glob("moedas/50/front/*")

sample_images_100_back = glob.glob("moedas/100/back/*")
sample_images_100_front = glob.glob("moedas/100/front/*")

# Lista com as características das imagens e suas respectivas classificações
data = []
answrData = []

# Extraindo características das moedas sozinhas
for i in sample_images_5_front:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.CINCO)

for i in sample_images_5_back:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.CINCO)

for i in sample_images_10_front:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.DEZ)

for i in sample_images_10_back:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.DEZ)

for i in sample_images_25_front:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.VINTECINCO)

for i in sample_images_25_back:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.VINTECINCO)

for i in sample_images_50_front:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.CINQUENTA)

for i in sample_images_50_back:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.CINQUENTA)

for i in sample_images_100_front:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.REAL)

for i in sample_images_100_back:
    data.append(calcHistFromFile(i))
    answrData.append(Coins.REAL)

print("--- %s seconds ---" % (time.time() - start_time))

--- 131.21533274650574 seconds ---


In [11]:
clf = RandomForestClassifier(n_estimators=100, random_state=1).fit(data,answrData)

In [13]:
import numpy as np
import cv2
cv = cv2
from matplotlib import pyplot as plt

def predictCoin(roi):
    hist = calcHist(roi, True)
    s = clf.predict([hist])
    
    return Coins[int(s)]

def findMultiplesCoin(imgCol):
    img = cv.cvtColor(imgCol, cv.COLOR_BGR2GRAY)
    img = improveEdges(img)
    img = cv2.blur(img, (3,3))

    # Single channel to multiples channel
    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

    circles = cv2.HoughCircles(img,method=cv2.HOUGH_GRADIENT,dp=1,minDist=116,
                               param1=60,param2=60,minRadius=50,maxRadius=90)
    return circles

# Imagem que será analisada
img = cv.imread("moedas/5/back/5-back (3).JPG")
d = 800 / img.shape[1]
dim = (800, int(img.shape[0] * d))
img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

# Cópia da imagem original que será apresentada no final
output = img.copy()

circles = findMultiplesCoin(img)

count = 0
if circles is not None:
    # coordinates and radii
    circles = np.round(circles[0,:]).astype(int)
    
    for (x, y, d) in circles:
        count += 1
                
        roi = img[y-d:y+d, x-d:x+d]
        
        roi = improveEdges(roi)
        
        coin = predictCoin(roi)
        
        cv.circle(output, (x,y), d, (0, 255, 0), 2)
        cv.putText(output, coin, (x - 40, y), cv2.FONT_HERSHEY_PLAIN,
                   1.5, (0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
                   
cv.imshow("Result", output)
cv.waitKey(0)
cv.destroyAllWindows()

In [12]:
def gravar_arquivo_arff(base_teste, classes):
    tam = len(base_teste[0][0])
    file = open('test.arff','w') 
 
    file.write('@relation teste\n') 
    for i in range(tam):
        file.write('@attribute '+ str(i) +' NUMERIC\n') 
    
    file.write('@attribute classes {')
    
    a = set(classes)
    
    for i in a:
        file.write(str(i)+',')
    
    file.write('}')    
    for i in range(tam):
         len(set(classes))
    
    file.write('\n@data\n') 

    for item in base_teste:
        for i in range(len(item[0])):
            file.write("%s," % str(item[0][i])) 
        file.write("%s\n" % item[1])    
 
    file.close() 
    print('arquivo gravado')
    
gravar_arquivo_arff(data,answrData)

arquivo gravado
